# Práctica NLP: Music Recommendation

Beatriz Torreiro Mosquera

### 1. Objetivo

El objetivo de esta práctica es la creación de un sistema de recomendación de música utilizando el modelo predictivo Word2Vec.

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

### 2. Conjunto de datos

El sistema de recomendación se va a realizar tomando como punto de partida las canciones de artistas de habla hispana recogidos en el dataset "hhgroups_merge_28_05.csv". Los campos incluidos en este dataset son: id, artista, cancion, album, letra, anyo, visitas.

 - Lectura del conjunto de datos

In [2]:
songs = pd.read_csv("data/hhgroups_merge_28_05.csv", header=0)
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219


In [3]:
songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9325 entries, 0 to 9324
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       9325 non-null   int64 
 1   artista  9325 non-null   object
 2   cancion  9325 non-null   object
 3   album    9325 non-null   object
 4   letra    9325 non-null   object
 5   anyo     9325 non-null   int64 
 6   visitas  9325 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 510.1+ KB


El dataset incluye información sobre 9325 canciones.

Si nos fijamos, la primera y la tercera canción, no contienen en el campo letra la información esperada, si no que aparece un mensaje indicando que no se posee esa información. El contenido exacto de la celda "letra" de la primera canción es el siguiente.

In [4]:
songs['letra'][0]

'Para su nuevo disco "Medicina", Denom ha vuelto a contar con la colaboración vocal de Jarfaiter, con quién ya ha colaborado anteriormente en el tema "Machete"; corte en el que tambien participa el grupo Gente Jodida. De la instrumental se encarga NarcosBeats.\n\n\n\n\n¿Tienes ya la letra para este tema? Ayúdanos y ¡Envíanosla!'

Como el campo "letra" va a ser el utilizado para la entrenar el modelo Word2Vec, es importante que este contenga información limpia y relevante. Por ese motivo se van a realizar los siguiente cambios en el dataset con el objetivo de limpiarlo:
   - Eliminar las entradas que no tienen letra.

In [5]:
songs = songs[~songs.letra.str.contains("¿Tienes ya la letra para este tema?")]

   - Eliminar de las letras todo el texto contenido entre paréntesis, ya que suele indicar que la estrofa siguiente es un estribillo o el artista que la canta.

In [6]:
songs['letra'] = songs['letra'].str.replace(r'\[(.*?)\]','')

 - Eliminar el contenido dentro de paréntesis, hace referencia al número de repeticiones de una estrofa.

In [7]:
songs['letra'] = songs['letra'].str.replace(r'\((.*?)\)','')

 - Eliminar el símbolo de retorno de carro "\n".

In [8]:
songs['letra'] = songs['letra'].str.replace(r'\n',' ')

 - Eliminar los símbolos extraños, nos quedamos sólo con las letras.

In [9]:
songs['letra'] = songs['letra'].str.replace(r'[^a-zA-ZáíéúóÁÉÍÓÚ ]','')

 - Se pasan todas las palabras a minúsculas para que no se cuenten por separado dos versiones de una misma palabra.

In [10]:
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,Yo que quería yo que pedía vida Se partió por...,2019,361
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,Mueve mueve mueve Ya no hay tiempo que perde...,2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto",Y en esta noche Quién me arropará Si el invie...,2019,219
5,5,Khan,Lobo,"Sin álbum, es un vídeo suelto",Que nadie creía en leyends hasta que llegué yo...,2019,683
6,6,Dyo,Claridad [Productor desconocido],"Sin álbum, es un tema suelto",Me olvidé tomarme la pastilla otra vez Estará ...,2019,58
